In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# parameters in the model 
q_lower = -20
q_upper = 20

### Model Explanation

**Modeling framework and notations**

- $dS_t = \sigma dW_t$, $W_t$ is a standard Brownian motion.
- $\delta^{a}, \delta^{b}$ denote the depth at which the agent posts LOs. 
- $M_t^b, M_t^a$ denote the Poisson processes corresponding to the arrival rate of **selling and buying** MOs, with intensity $\lambda^b, \lambda^a$, respectively. 
- $N_t^b, N_t^a$ denote the counting processes for the MM's filled buy and sell LOs. 
- conditional on a MO arrival, the posted LO is filled with probability $e^{-\kappa \delta}$
- $dQ_t = dN_t^b - dN_t^a$ denotes the process of inventory, inventory has upper and lower bounds, i.e. $\underline{q} \leq q \leq \bar{q}$. 
- $dX_t = (S_t + \delta_t^a) dN_t^a - (S_t - \delta_t^b) dN_t^b $ denotes the cash process. 

**Performance criterion**
$$ H^{\delta}(t,x,S,q) =  \mathbb{E}_{t,x,q,S} \big[ X_T + Q_T^{\delta} (S_T^{\delta} - \alpha Q_T^{\delta}) - \phi \int_t^T (Q_u)^2 du \big]$$


**Value function:**

$$H(t,x,S,q) = \sup_{\delta^{\pm} \in \mathcal{A}} H^{\delta}(t,x,S,q) = \sup_{\delta^{\pm} \in \mathcal{A}} \mathbb{E}_{t,x,q,S} \big[ X_T + Q_T^{\delta} (S_T^{\delta} - \alpha Q_T^{\delta}) - \phi \int_t^T (Q_u)^2 du \big]$$

with terminal condition $H(T,x,S,q) = x + q(S - \alpha q)$.

**HJB equation**
$$ 0 = \partial_t H(t,x,S,q) + \frac{1}{2} \sigma^2 \partial^2_{SS}H(t,x,S,q) - \phi q^2 + \lambda^{a}\sup_{\delta^a} \big[ e^{-\kappa^{a}\delta^{a}} (H(t,x+S+\delta^{a},S,q-1)-H(t,x,S,q)) \big] \mathbb{1}_{q>\underline{q}} + \lambda^{b}\sup_{\delta^b} \big[ e^{-\kappa^{b}\delta^{b}} (H(t,x-S+\delta^{b},S,q+1)-H(t,x,S,q)) \big] \mathbb{1}_{q<\bar{q}}$$

**Solve HJB equation:**

Optimal depths:
$$\delta^{a,*}(t,q) = \frac{1}{\kappa^{a}} - h(t,q-1) + h(t,q) , \quad q \neq \underline{q}$$
$$\delta^{b,*}(t,q) = \frac{1}{\kappa^{b}} - h(t,q+1) + h(t,q), \quad q \neq \bar{q}$$

Assuming $\kappa = \kappa^{a} = \kappa^{b}$ and $h(t,q) = \frac{1}{\kappa} \log \omega(t,q)$. Stack $\omega(t,q)$ into a vector $\boldsymbol{\omega}(t) = [\omega(t,\bar{q}),\omega(t,\bar{q}-1), ..., \omega(t,\underline{q})]$. Let $\boldsymbol{A}$ denote a $(\bar{q} - \underline{q} + 1)$ - square matrix with:

- $\boldsymbol{A}_{q,q} = -\phi \kappa q^2$
- $\boldsymbol{A}_{q-1,q} = \lambda^{a} e^{-1}$
- $\boldsymbol{A}_{q+1,q} = \lambda^{b} e^{-1}$

Finally, we have $$\boldsymbol{\omega}(t) = e^{\boldsymbol{A}(T-t)} \boldsymbol{z}$$, where $\boldsymbol{z}$ is a vector with $z_j = e^{-\alpha \kappa j^2}$

Optimal depths:
$$\delta^{a,*}(t,q) = \frac{1}{\kappa} + \frac{1}{\kappa} \log \frac{\omega(t,q)}{\omega(t,q-1)}, \quad q \neq \underline{q}$$
$$\delta^{b,*}(t,q) = \frac{1}{\kappa} + \frac{1}{\kappa} \log \frac{\omega(t,q)}{\omega(t,q+1)}, \quad q \neq \bar{q}$$

### Performance of Simple Market Making Simulation

Run 80 MC simulations

In [ ]:
data = pd.read_csv('Agent_simple_MMmodel.csv')
simulation = list(data.groupby(['Iteration']))
iteration = len(simulation)

In [ ]:
fig,ax = plt.subplots(3,2,figsize=(20,20))

# Plot position/midprice/general pnl (with fee) of lp/realised pnl of lp
for i in range(iteration):
    ax[0,0].plot(simulation[i][1]['Time Step'], simulation[i][1]['LP: Position'],'--')
ax[0,0].set_title('LP: Position')
ax[0,0].set_xlabel("Time step")
ax[0,0].set_ylabel("Position")
ax[0,0].set_ylim([q_lower,q_upper])
ax[0,0].set_yticks([i for i in range(q_lower,q_upper+1,4)])

for i in range(iteration):
    ax[1,0].plot(simulation[i][1]['Time Step'], simulation[i][1]['Market Sate'])
ax[1,0].set_title('Market Trading Mode')
ax[1,0].set_xlabel("Time step")
ax[1,0].set_ylabel("Trading Mode")

for i in range(iteration):
    ax[0,1].plot(simulation[i][1]['Time Step'], simulation[i][1]['External Midprice'])
ax[0,1].set_title('Midprice')
ax[0,1].set_xlabel("Time step")
ax[0,1].set_ylabel("Price")

for i in range(iteration):
    ax[1,1].plot(simulation[i][1]['Time Step'], simulation[i][1]['LP: General Pnl'])
ax[1,1].set_title('LP: General Pnl with fee over time')
ax[1,1].set_xlabel("Time step")
ax[1,1].set_ylabel("Pnl")

# Plot infrastructure fee (related to average trading volume)
for i in range(iteration):
    ax[2,0].plot(simulation[i][1]['Time Step'], simulation[i][1]['InfrafeeAccount'])
ax[2,0].set_title('Infrastructure Fee Account')
ax[2,0].set_xlabel("Time step")
ax[2,0].set_ylabel("Fee")


# Plot margin account of lp
for i in range(iteration):
    ax[2,1].plot(simulation[i][1]['Time Step'], simulation[i][1]['LP: Margin Account'])
ax[2,1].set_title('LP: Margin Account')
ax[2,1].set_xlabel("Time step")
ax[2,1].set_ylabel("Margin")


plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# parameters in the model 
q_lower = -20
q_upper = 20

data_0p = pd.read_csv('MC_informedtrader_0p.csv')
data_25p = pd.read_csv('MC_informedtrader_25p.csv')
data_50p = pd.read_csv('MC_informedtrader_50p.csv')
data_75p = pd.read_csv('MC_informedtrader_75p.csv')
data_100p = pd.read_csv('MC_informedtrader_100p.csv')

ave_0p = data_0p.groupby(['Time Step']).mean()
ave_25p = data_25p.groupby(['Time Step']).mean()
ave_50p = data_50p.groupby(['Time Step']).mean()
ave_75p = data_75p.groupby(['Time Step']).mean()
ave_100p = data_100p.groupby(['Time Step']).mean()

label = [
    '0% vol from Info_trader',
    '25% vol from Info_trader',
    '50% vol from Info_trader',
    '75% vol from Info_trader',
    '100% vol from Info_trader',
]

In [ ]:
fig,ax = plt.subplots(4,2,figsize=(20,25))

# Plot one of simulations of LP_position and informed_trder_position
ax[0,0].plot(ave_0p.index, data_0p[data_0p['Iteration'] == 1]['LP: Position'],'--o',label=label[0])
ax[0,0].plot(ave_25p.index, data_25p[data_25p['Iteration'] == 1]['LP: Position'],'--o',label=label[1])
ax[0,0].plot(ave_50p.index, data_50p[data_50p['Iteration'] == 1]['LP: Position'],'--o',label=label[2])
ax[0,0].plot(ave_75p.index, data_75p[data_75p['Iteration'] == 1]['LP: Position'],'--o',label=label[3])
ax[0,0].plot(ave_100p.index, data_100p[data_100p['Iteration'] == 1]['LP: Position'],'--o',label=label[4])
ax[0,0].set_title('LP: Position')
ax[0,0].set_xlabel("Time step")
ax[0,0].set_ylabel("Position")
ax[0,0].set_ylim([q_lower,q_upper])
ax[0,0].set_yticks([i for i in range(q_lower,q_upper+1,4)])
ax[0,0].legend()

ax[0,1].plot(ave_0p.index, data_0p[data_0p['Iteration'] == 1]['Trader: Position'],'--o',label=label[0])
ax[0,1].plot(ave_25p.index, data_25p[data_25p['Iteration'] == 1]['Trader: Position'],'--o',label=label[1])
ax[0,1].plot(ave_50p.index, data_50p[data_50p['Iteration'] == 1]['Trader: Position'],'--o',label=label[2])
ax[0,1].plot(ave_75p.index, data_75p[data_75p['Iteration'] == 1]['Trader: Position'],'--o',label=label[3])
ax[0,1].plot(ave_100p.index, data_100p[data_100p['Iteration'] == 1]['Trader: Position'],'--o',label=label[4])
ax[0,1].set_title('LP: Position')
ax[0,1].set_xlabel("Time step")
ax[0,1].set_ylabel("Position")
ax[0,1].set_ylim([q_lower,q_upper])
ax[0,1].set_yticks([i for i in range(q_lower,q_upper+1,4)])
ax[0,1].legend()

# Plot average realised Pnl for LP/ informed trader
ax[1,0].plot(ave_0p.index, ave_0p['LP: RealisedPnl'],label=label[0])
ax[1,0].plot(ave_25p.index, ave_25p['LP: RealisedPnl'],label=label[1])
ax[1,0].plot(ave_50p.index, ave_50p['LP: RealisedPnl'],label=label[2])
ax[1,0].plot(ave_75p.index, ave_75p['LP: RealisedPnl'],label=label[3])
ax[1,0].plot(ave_100p.index, ave_100p['LP: RealisedPnl'],label=label[4])
ax[1,0].set_title('LP: Realised Pnl over time')
ax[1,0].set_xlabel("Time step")
ax[1,0].set_ylabel("Pnl")
ax[1,0].legend()

ax[1,1].plot(ave_0p.index, ave_0p['Trader: RealisedPnl'],label=label[0])
ax[1,1].plot(ave_25p.index, ave_25p['Trader: RealisedPnl'],label=label[1])
ax[1,1].plot(ave_50p.index, ave_50p['Trader: RealisedPnl'],label=label[2])
ax[1,1].plot(ave_75p.index, ave_75p['Trader: RealisedPnl'],label=label[3])
ax[1,1].plot(ave_100p.index, ave_100p['Trader: RealisedPnl'],label=label[4])
ax[1,1].set_title('Trader: Realised Pnl over time')
ax[1,1].set_xlabel("Time step")
ax[1,1].set_ylabel("Pnl")
ax[1,1].legend()

# Plot average general Pnl (with fee) for LP/ informed trader
ax[2,0].plot(ave_0p.index, ave_0p['LP: GeneralPnl'],label=label[0])
ax[2,0].plot(ave_25p.index, ave_25p['LP: GeneralPnl'],label=label[1])
ax[2,0].plot(ave_50p.index, ave_50p['LP: GeneralPnl'],label=label[2])
ax[2,0].plot(ave_75p.index, ave_75p['LP: GeneralPnl'],label=label[3])
ax[2,0].plot(ave_100p.index, ave_100p['LP: GeneralPnl'],label=label[4])
ax[2,0].set_title('LP: General Pnl (with fee) over time')
ax[2,0].set_xlabel("Time step")
ax[2,0].set_ylabel("Pnl")
ax[2,0].legend()

ax[2,1].plot(ave_0p.index, ave_0p['Trader: GeneralPnl'],label=label[0])
ax[2,1].plot(ave_25p.index, ave_25p['Trader: GeneralPnl'],label=label[1])
ax[2,1].plot(ave_50p.index, ave_50p['Trader: GeneralPnl'],label=label[2])
ax[2,1].plot(ave_75p.index, ave_75p['Trader: GeneralPnl'],label=label[3])
ax[2,1].plot(ave_100p.index, ave_100p['Trader: GeneralPnl'],label=label[4])
ax[2,1].set_title('Trader: General Pnl (with fee) over time')
ax[2,1].set_xlabel("Time step")
ax[2,1].set_ylabel("Pnl")
ax[2,1].legend()

# Plot average margin account for LP/ informed trader
ax[3,0].plot(ave_0p.index, ave_0p['LP: Margin Account'],'--o',label=label[0])
ax[3,0].plot(ave_25p.index, ave_25p['LP: Margin Account'],'--o',label=label[1])
ax[3,0].plot(ave_50p.index, ave_50p['LP: Margin Account'],'--o',label=label[2])
ax[3,0].plot(ave_75p.index, ave_75p['LP: Margin Account'],'--o',label=label[3])
ax[3,0].plot(ave_100p.index, ave_100p['LP: Margin Account'],'--o',label=label[4])
ax[3,0].set_title('LP: Margin Account over time')
ax[3,0].set_xlabel("Time step")
ax[3,0].set_ylabel("Pnl")
ax[3,0].legend()

ax[3,1].plot(ave_0p.index, ave_0p['Trader: Margin Account'],'--o',label=label[0])
ax[3,1].plot(ave_25p.index, ave_25p['Trader: Margin Account'],'--o',label=label[1])
ax[3,1].plot(ave_50p.index, ave_50p['Trader: Margin Account'],'--o',label=label[2])
ax[3,1].plot(ave_75p.index, ave_75p['Trader: Margin Account'],'--o',label=label[3])
ax[3,1].plot(ave_100p.index, ave_100p['Trader: Margin Account'],'--o',label=label[4])
ax[3,1].set_title('Trader: Margin Account over time')
ax[3,1].set_xlabel("Time step")
ax[3,1].set_ylabel("Margin")
ax[3,1].legend()

plt.show()


In [ ]:
data_0p['LP: Position'] = np.abs(data_0p['LP: Position'])
data_0p['Trader: Position'] = np.abs(data_0p['Trader: Position'])
data_25p['LP: Position'] = np.abs(data_25p['LP: Position'])
data_25p['Trader: Position'] = np.abs(data_25p['Trader: Position'])
data_50p['LP: Position'] = np.abs(data_50p['LP: Position'])
data_50p['Trader: Position'] = np.abs(data_50p['Trader: Position'])
data_75p['LP: Position'] = np.abs(data_75p['LP: Position'])
data_75p['Trader: Position'] = np.abs(data_75p['Trader: Position'])
data_100p['LP: Position'] = np.abs(data_100p['LP: Position'])
data_100p['Trader: Position'] = np.abs(data_100p['Trader: Position'])

ave_0p = data_0p.groupby(['Time Step']).mean()
ave_25p = data_25p.groupby(['Time Step']).mean()
ave_50p = data_50p.groupby(['Time Step']).mean()
ave_75p = data_75p.groupby(['Time Step']).mean()
ave_100p = data_100p.groupby(['Time Step']).mean()